In [307]:
import numpy as np
import pandas as pd
import os

%matplotlib inline

#user specify instrument and contract month

product=''
month=''

while True:
    inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
    if inst==1:
        product='Brent'
        break
    elif inst==2:
        product='Gasoil'
        break   
    elif inst==3:
        product='HO'
        break      
    elif inst==4:
        product='RBOB'
        break       
    elif inst==5:
        product='WTI'
        break       
    else:
        print 'You did not input a correct number. Please try again.'

while True:
    Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
    if Mth==1:
        month='Jan'
        break
    elif Mth==2:
        month='Feb'
        break   
    elif Mth==3:
        month='Mar'
        break   
    elif Mth==4:
        month='Apr'
        break   
    elif Mth==5:
        month='May'
        break          
    elif Mth==6:
        month='Jun'
        break   
    elif Mth==7:
        month='Jul'
        break   
    elif Mth==8:
        month='Aug'
        break   
    elif Mth==9:
        month='Sep'
        break   
    elif Mth==10:
        month='Oct'
        break   
    elif Mth==11:
        month='Nov'
        break   
    elif Mth==12:
        month='Dec'
        break 
    else:
        print 'You did not input a correct number. Please try again.'
        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 3
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7


In [308]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [309]:
cd $product

C:\Users\boonsin.teo\Documents\Quandl\HO


In [310]:
cd $month

C:\Users\boonsin.teo\Documents\Quandl\HO\Jul


In [311]:

count=0
path = os.getcwd()

#indicate evaluation dates

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')

i=int(raw_input('Please input 1 if seasonal period start on same year as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')

j=int(raw_input('Please input 1 if seasonal period ends on same year as contract year and 0 if not: '))

while True:
    pos=1
    a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
    if a=='y':
        break
    if a=='n':
        pos=pos*(-1)
        break
    else:
        print "Please input either 'y' or 'n' for position"

print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

listing = os.listdir(path)
for infile in listing:
    temp=pd.read_excel(infile) #read each file into panda dataframe
    
    contract_yr_str=infile[-8:-4] # get the contract year
    
    seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
    seasonal_year=str(seasonal_year) # convert to string
    
    start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
    end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
    
    dates=pd.date_range(start_date, end_date) # range of dates for date axis
    dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
    sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
    dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
    dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
    temp=temp.set_index('Date') # set index as Date to facilitate joining
    
    temp=dates_dataframe.join(temp) # join dataframes together on Date
   

    temp.reset_index(inplace=True)
    
    temp=temp[temp['Date']>=start_date] # filter for start date
    #temp.reset_index(inplace=True)
    temp=temp[temp['Date']<=end_date] # filter for end date
    #temp=temp.iloc[0:250]

    temp=temp.set_index('Date') # creating new index as Date
    
    
    temp=temp.reset_index() # putting date back out into col
    temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d')
    temp=temp.set_index('Date')
    temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # becomes a series, hwo to solve this again?

    #combine into seasonal index table
    
    if count==0:
        Seasonal_Combi=temp
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi.columns=[seasonal_year]
        
        count+=1
        
    else:
        
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi[seasonal_year]=temp['Settle']

#remove columns that are all NA
Seasonal_Combi.dropna(axis=1,how='all',inplace=True)

#remove rows that are all NA
Seasonal_Combi.dropna(axis=0,how='all',inplace=True)

#fillna values with values of forward value
Seasonal_Combi.fillna(method='bfill',inplace=True)
#fillna values with values of back value
Seasonal_Combi.fillna(method='ffill',inplace=True)

#Items to Create
#PnL - use last value vs first value

z=1
factor=1000
if product =='RBOB':
    z=42
elif product=='HO':
    z=42
elif product=='Gasoil':
    factor=100
else:
    z=1
    factor=1000

pnl_series=(Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])*z*pos*factor    

# Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100
    
#Largest Drawdown - use max and min
max_series=Seasonal_Combi.max()
min_series=Seasonal_Combi.min()
if pos==1:
    max_drawdown=(min_series-Seasonal_Combi.iloc[0])*pos*z*factor
else:
    max_drawdown=(max_series-Seasonal_Combi.iloc[0])*pos*z*factor
    

#Plot Equity Curve - use plot below for all columns
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])/(Seasonal_Combi.iloc[0]))



"""  
#plot on line chart
ax=Seasonal_Combi.plot.line(x=Seasonal_Combi.index, y='SI_avg_rolling',figsize=(20,5),xticks=[x for x in range(0,250,10)],title=month+" "+product)
ax.set_xlabel('Date')
ax.set_ylabel('Seasonal Index')
ax.legend(loc=2)
"""

Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 18
Please input 1 if seasonal period start on same year as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 01
Please input end day of seasonal period in DD format: 31
Please input 1 if seasonal period ends on same year as contract year and 0 if not: 1
Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


'  \n#plot on line chart\nax=Seasonal_Combi.plot.line(x=Seasonal_Combi.index, y=\'SI_avg_rolling\',figsize=(20,5),xticks=[x for x in range(0,250,10)],title=month+" "+product)\nax.set_xlabel(\'Date\')\nax.set_ylabel(\'Seasonal Index\')\nax.legend(loc=2)\n'

In [312]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\HO


In [313]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [314]:
cd ..

C:\Users\boonsin.teo\Documents


In [315]:
Seasonal_Combi.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Date,,,,,,,,,,,,,,,,,,,,,
01/18,0.4771,0.5912,0.4815,0.3527,0.6261,0.7060,0.5291,0.7538,0.8375,1.2208,...,2.4074,1.4552,2.1080,2.6395,2.9850,3.0301,2.8949,1.6045,0.9999,1.6611
01/19,0.4741,0.5912,0.4815,0.3527,0.6304,0.7324,0.5336,0.7538,0.8375,1.2208,...,2.3858,1.4552,2.1080,2.6434,2.9947,3.0363,2.8949,1.6045,0.9999,1.6722
01/20,0.4705,0.5912,0.4815,0.3439,0.6279,0.7231,0.5336,0.7538,0.8375,1.2140,...,2.3858,1.4552,2.0826,2.6128,2.9485,3.0363,2.8949,1.6045,0.9572,1.7035
01/21,0.4705,0.5876,0.4795,0.3519,0.6111,0.7231,0.5336,0.7538,0.8335,1.2475,...,2.3858,1.4632,2.0420,2.6421,2.9612,3.0363,2.8949,1.6305,0.9883,1.6908
01/22,0.4705,0.5914,0.4748,0.3598,0.6248,0.7231,0.5336,0.7529,0.8428,1.2615,...,2.3858,1.4346,1.9970,2.6196,2.9612,3.0363,2.9235,1.6210,1.0767,1.6908


In [316]:
pnl_series

1996    -315.0
1997    -180.6
1998     571.2
1999     294.0
2000    -113.4
2001    -499.8
2002     693.0
2003    1293.6
2004   -1890.0
2005    1856.4
2006    3364.2
2007    6283.2
2008    2423.4
2009    1377.6
2010   -6384.0
2011    4334.4
2012     205.8
2013    3662.4
2014     -75.6
2015    4552.8
2016    6648.6
2017     193.2
dtype: float64

In [317]:
percent_return

1996   -0.015720
1997   -0.007273
1998    0.028245
1999    0.019847
2000   -0.004312
2001   -0.016856
2002    0.031185
2003    0.040860
2004   -0.053731
2005    0.036206
2006    0.043455
2007    0.096535
2008    0.023968
2009    0.022540
2010   -0.072106
2011    0.039098
2012    0.001642
2013    0.028778
2014   -0.000622
2015    0.067560
2016    0.158316
2017    0.002769
dtype: float64

In [318]:
max_drawdown

1996    -638.4
1997    -390.6
1998    -567.0
1999    -369.6
2000    -630.0
2001    -499.8
2002       0.0
2003    -256.2
2004   -2079.0
2005    -285.6
2006       0.0
2007       0.0
2008   -3070.2
2009    -865.2
2010   -6384.0
2011   -1911.0
2012   -1533.0
2013       0.0
2014     -75.6
2015       0.0
2016   -1793.4
2017       0.0
dtype: float64

In [319]:
print 'The win percentage is %f percent' %Win_Percent

The win percentage is 68.181818 percent


In [320]:
pwd

u'C:\\Users\\boonsin.teo\\Documents'